In [2]:
import re

In [11]:
inp = """ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""

In [12]:
keys = ["byr:","iyr","eyr:","hgt:","hcl:","ecl:","pid:","cid:"]

In [13]:
def parse(inp):
    return [l.strip().replace("\n", " ") for l in inp.split('\n\n')]

def part1(passports):
    res = 0
    r = re.compile("([a-z]{3}:)")

    for p in passports:
        not_found = list(filter(lambda k: k not in p, keys))
        found = r.findall(p)
        assert(len(found) + len(not_found) == 8)
        if len(not_found) == 0 or (len(not_found) == 1 and not_found[0] == "cid:"):
            res += 1

    return res

part1(parse(inp))
        

2

In [14]:
with open('data/04.txt') as f:
    inp = f.read().strip()

part1(parse(inp))

228

In [17]:
def parse2(inp):
    res = []
    r = re.compile("([a-z]{3}:\S+)")
    for l in inp.split('\n\n'):
        pp = {}
        for f in r.findall(l):
            v = f.split(':')
            pp[v[0]] = v[1]
        res.append(pp)
    return res


In [18]:
all_invalid = """eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007"""

all_valid = """pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719"""

In [19]:
def val_hgt(v: str):
    if v.endswith("cm"):
        return 150 <= int(v[:-2]) <= 193
    elif v.endswith("in"):
        return 59 <= int(v[:-2]) <= 76
    return False

In [21]:
fields = {
    "byr": lambda v: 1920 <= int(v) <= 2002,
    "iyr": lambda v: 2010 <= int(v) <= 2020,
    "eyr": lambda v: 2020 <= int(v) <= 2030,
    "hgt": val_hgt,
    "hcl": lambda v: re.match("^#[0-9a-f]{6}$", v) is not None,
    "ecl": lambda v: v in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"],
    "pid": lambda v: re.match("^[0-9]{9}$", v) is not None,
    "cid": lambda v: True
}
def part2(passports):
    res = 0

    for p in passports:
        not_found = list(filter(lambda k: k not in p.keys(), fields.keys()))
        # print(not_found)
        if len(not_found) == 0 or (len(not_found) == 1 and not_found[0] == "cid"):
            p["cid"] = 0
            if sum(val(p[field]) for field, val in fields.items()) == 8:
                res += 1

    return res

part2(parse2(inp))

175

In [22]:
assert(fields["byr"]("2002"))
assert(fields["byr"]("2003") == False)

assert(fields["hgt"]("60in"))
assert(fields["hgt"]("190cm"))
assert(fields["hgt"]("190in") == False)
assert(fields["hgt"]("190") == False)

In [23]:
assert(fields["hcl"]("#123abc"))
assert(fields["hcl"]("#123abz") == False)
assert(fields["hcl"]("123abc") == False)

assert(fields["ecl"]("brn"))
assert(fields["ecl"]("wat") == False)

assert(fields["pid"]("000000001"))
assert(fields["pid"]("0123456789") == False)